In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import scatter

import pathpyG as pp
pp.config['torch']['device'] = 'cpu'

In [2]:
def compute_weighted_outdegrees(graph):
    weighted_outdegree = scatter(graph.data.edge_weight, graph.data.edge_index[0], dim=0, dim_size=graph.data.num_nodes, reduce='sum')
    return weighted_outdegree

def compute_transition_probabilities(graph):
    weighted_outdegree = compute_weighted_outdegrees(graph)
    source_ids = graph.data.edge_index[0]
    return graph.data.edge_weight/ weighted_outdegree[source_ids]


In [3]:
from torch.utils.data import Dataset
import torch

# https://en.wikipedia.org/wiki/Pairing_function
# https://math.stackexchange.com/questions/1377929/generalization-of-cantor-pairing-function-to-triples-and-n-tuples
def cantor_pairing(x, y):
    """
    Computes the Cantor pairing value for two integers x and y.
    The Cantor pairing maps two integers to a unique integer.

    Args:
        x (int): The first integer.
        y (int): The second integer.

    Returns:
        int: The Cantor pairing value for the given integers x and y.
    """
    return (x + y) * (x + y + 1) // 2 + y

def cantor_encode_tensor(tnsr):
    """
    Encodes a list of integers in tensor rows into single integers using Cantor pairing.
    The function recursively applies the Cantor pairing function to pairs of elements
    in the input tensor until it encodes the entire list into a single integer.

    If the input tensor is empty, the function returns 0.

    Args:
        tnsr (torch.Tensor): A tensor containing a list of integers.

    Returns:
        torch.Tensor: The Cantor encoded integer representing the input list.
    """
    if tnsr.size(1) == 0:
        # Termination point. The added 0 has no effect on the returned integer
        return torch.tensor(0)
    else:
        return cantor_pairing(tnsr[:, 0], cantor_encode_tensor(tnsr[:, 1:]))


class WalksDataset(Dataset):
    """
    Dataset class to handle sequences of node walks.

    Args:
        dag_data (object): The input DAG data.
        dict_cantor_to_honode_ixs_mapping (dict): Dictionary mapping Cantor encoded indices to higher-order node indices.
        max_order (int): Maximum order of nodes in a walk.
    """

    def __init__(self, dag_data, dict_cantor_to_honode_ixs_mapping, max_order):
        self.max_order = max_order
        self.dict_cantor_to_honode_ixs_mapping = dict_cantor_to_honode_ixs_mapping
        self._preprocess_data(dag_data.dags)
        self._create_tensors_and_encodings()

    def _preprocess_data(self, dags):
        """
        Preprocesses the DAG data to extract walks and their counts.

        Args:
            dags DAGData object
        """
        self.walks_by_length = {}
        self.walk_counts_by_length = {}
        self.total_sequences = 0
        
        for dag in dags:
            node_seq_path = dag.node_sequence.T[0]
            seq_length = len(node_seq_path)
            if seq_length not in self.walks_by_length:
                self.walks_by_length[seq_length] = []
                self.walk_counts_by_length[seq_length] = []
            self.walks_by_length[seq_length].append(node_seq_path)
            # Answer to Moritz question
            # probably one weight per DAG would be enough
            self.walk_counts_by_length[seq_length].append(int(dag.edge_weight.unique()))
            self.total_sequences += 1

    def _create_tensors_and_encodings(self):
        """
        Creates tensors and encodings for the walks.
        """
        self.walk_tensors_by_length = {
            length: torch.stack(walks, dim=0)
            for length, walks in self.walks_by_length.items()
        }
        self.bipartite_encoded_walks_by_length = {
            length: self._bipartite_encode(length)
            for length in self.walk_tensors_by_length
        }

    def _bipartite_encode(self, walk_length):
        """
        Encodes the walks in bipartite form, i.e.,
        representing transitions between indexes of higher-order nodes.

        In bipartite encoding, each transition in the walk sequence corresponds to a pair of indexes,
        where the first `self.max_order` transitions utilize indexes from the higher-order nodes of the i-th order tensors, and the subsequent transitions use indexes of the `self.max_order` most recent nodes.

        Args:
            walk_length (int): The length of the walk.

        Returns:
            torch.Tensor: Bipartite-encoded walk sequences.
        """

        list_cantor_node_ixs_tensors = []
        
        for i in range(1, walk_length + 1):
            hon_ixs_tensor = self.walk_tensors_by_length[walk_length][:, max(0, i - self.max_order):i]
            cantor_encoded = cantor_encode_tensor(hon_ixs_tensor)
            mapped_indices = cantor_encoded.apply_(self.dict_cantor_to_honode_ixs_mapping[min(i, self.max_order)].get)
            list_cantor_node_ixs_tensors.append(mapped_indices)
            
        return torch.stack(list_cantor_node_ixs_tensors, dim=1)

    def __getitem__(self, index_tuple):
        """
        Retrieves a bipartite-encoded node sequence from the dataset.

        Args:
            index_tuple (tuple): A tuple containing the length of the sequence (int) and the index of the sequence within that length (int).

        Returns:
            torch.Tensor: The bipartite-encoded node sequence.
        """
        # This cannot work with batching. Could work if each walk length had its own Dataset object
        walk_length, index = index_tuple
        return self.bipartite_encoded_walks_by_length[walk_length][index]

    def __len__(self):
        """
        Returns the total number of sequences in the dataset.

        Returns:
            int: Total number of sequences.
        """
        return self.total_sequences


In [4]:
dag_data = pp.DAGData(pp.IndexMap(list("01234")))

dag_data.append_walk(list("0230230"), weight=30)
dag_data.append_walk(list("1241241"), weight=70)
dag_data.append_walk(list("0230241"), weight=1)
#
# dag_data.append_walk(list("0230230230230230230"), weight=30)


m = pp.MultiOrderModel.from_DAGs(dag_data, max_order=2)


In [5]:
def get_cantor_econding_mon_nodes(m):
    """ 
    Applies cantor encoding to the node sequences of the higher-order nodes for all the layers of a MultiOrderModel
    Returns a dictionary containing, for each layer, a mapping from the cantor index (encoding) of the sequence to the index of the ho-node in the Graph object
    """
    dict_cantor_to_honode_ixs_mapping = {}
    # This works cause the ho-node sequences are sorted by their indices
    for order, hon in m.layers.items():
        cantor_ids = cantor_encode_tensor(hon.data.node_sequence)
        cantor_to_node_ixs_mapping = {cantor_id.item(): i for i, cantor_id in enumerate(cantor_ids)}
        dict_cantor_to_honode_ixs_mapping[order] = cantor_to_node_ixs_mapping
    return dict_cantor_to_honode_ixs_mapping
dict_cantor_to_honode_ixs_mapping = get_cantor_econding_mon_nodes(m)

In [6]:
walk_data = WalksDataset(dag_data, dict_cantor_to_honode_ixs_mapping, max_order=2)

TODO: following cells contraints all this to be done on a single value of path length

In [7]:
l = 7
source_to_target_from_walks = walk_data.bipartite_encoded_walks_by_length[l]
path_counts = walk_data.walk_counts_by_length[l]

In [8]:
# FIND ANOTHER WAY TO GET NODE COUNTS
from torch_geometric.loader import DataLoader
dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(dag_data.dags)))).to(pp.config["torch"]["device"])


In [9]:
unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
node_emission_probabilities = counts / counts.sum()

source_to_target_edge_index_zeroth = torch.stack([
    torch.zeros_like(source_to_target_from_walks[:, 0]),
    source_to_target_from_walks[:, 0]
])

# log likelihood
tot_log_lh = 0
# log likelihood for the 0-th steps
lh_l = torch.mul(torch.log(node_emission_probabilities[source_to_target_edge_index_zeroth[1]]), torch.tensor(path_counts))
tot_log_lh += lh_l.sum()

for i in range(0, l - 1):
    T = compute_transition_probabilities(m.layers[min(i + 1, walk_data.max_order)])
    # Prepare source_to_target_edge_index
    source_to_target_edge_index = source_to_target_from_walks[:, i:i + 2].T.squeeze()
    # log likelihood for i-th steps
    lh_l = torch.mul(torch.log(T[source_to_target_edge_index[1]]), torch.tensor(path_counts))
    tot_log_lh += lh_l.sum()

tot_log_lh


tensor(-449.9671)

- Adapt to multiple walk lengths
- Better structure for package
- Degrees of freedom etc (actually making the likilhood ration test test)
    - This all thing above is getting lh (organize)
    - Need degrees of freedom
    - Need to actually perform che likelihood ratio test (see line 407 of this: https://github.com/IngoScholtes/pathpy/blob/master/pathpy/MultiOrderModel.py#L378). 
- test functions 
    - Dof computation
    - lh computation
    - encoding


In [10]:
def get_mon_likelihood(mon, walk_data):

    dict_cantor_to_honode_ixs = get_cantor_econding_mon_nodes(mon)
    inner_walk_data = WalksDataset(walk_data, dict_cantor_to_honode_ixs, max_order=max(mon.layers))

    # zeroth order part:
    # TODO: FIND ANOTHER WAY TO GET NODE COUNTS for node emission probabilities ###########
    # notice that this is building the zeroth order of a MultiOrderModel
    from torch_geometric.loader import DataLoader
    dag_graph = next(iter(DataLoader(dag_data.dags, batch_size=len(walk_data.dags)))).to(pp.config["torch"]["device"])
    __unique_nodes, counts = torch.unique(dag_graph.node_sequence, return_counts=True)
    node_emission_probabilities = counts / counts.sum()
    #####################
    # at the zeroth order, source_to_target are just connections from start node to any of the first order nodes
    source_to_target_edge_index_zeroth = torch.stack([
        torch.zeros_like(source_to_target_from_walks[:, 0]),
        source_to_target_from_walks[:, 0]
    ])

    # Beginnign of log likelihood computation
    tot_log_lh = 0
    for walk_length in inner_walk_data.walks_by_length:
        # log likelihood for the 0-th order
        path_counts = inner_walk_data.walk_counts_by_length[walk_length]
        lh_l = torch.mul(torch.log(node_emission_probabilities[source_to_target_edge_index_zeroth[1]]), torch.tensor(path_counts))
        tot_log_lh += lh_l.sum()

        # TODO: modify name 'source_to_target'. These are paths represented with the indexes of mon nodes 
        source_to_target = inner_walk_data.bipartite_encoded_walks_by_length[l]
        for i in range(0, l - 1):
            T = compute_transition_probabilities(m.layers[min(i + 1, inner_walk_data.max_order)])
            # Prepare source_to_target_edge_index
            source_to_target_edge_index = source_to_target[:, i:i + 2].T.squeeze()
            # log likelihood for i-th steps
            lh_l = torch.mul(torch.log(T[source_to_target_edge_index[1]]), torch.tensor(path_counts))
            tot_log_lh += lh_l.sum()
    return tot_log_lh

get_mon_likelihood(m, dag_data)

tensor(-449.9671)

In [11]:


def get_mon_dof(m,max_order, assumption = "paths"):
    """
    The degrees of freedom fo the kth layer of a multi-order model this depende on the number of different paths of exactly length k in the graph.
    Therefore, we can obtain this values by summing the entries of the kth power of the binary adhacency matrix of the graph.
    Finally, we must consider that, due the conservation of probablility, all non-zero rows of the transition matrix of the higher-order network must sum to one. 
    This poses on additional constraint per row that respects the condition, which should be removed from the total count of degrees of freedom.

    @param: maxOrder: the maximum order up to which model layers shall be
    taken into account

    @param assumption: if set to 'paths', for the degree of freedom calculation
        only paths in the first-order network topology will be considered. This is
        needed whenever we model paths in a *given* network topology.
        If set to 'ngrams' all possible n-grams will be considered, independent of whether they
        are valid paths in the first-order network or not. The 'ngrams' and the 'paths' assumption
        coincide if the first-order network is fully connected, i.e. if all possible paths actually occur.
    """
    if max_order == None:
        max_order = max(m.layers)
    assert max_order <= max(m.layers), 'Error: maxOrder cannot be larger than maximum order of multi-order network'

    # for both assumptions, the zeroth order simply gives the node probabilities. 
    dof = m.layers[1].data.num_nodes - 1

    if assumption == "paths":
        for order in range(1, max_order+1):
            num_order_paths = m.layers[order].data.num_nodes
            num_nonzero_outdegrees = len(m.layers[order].data.edge_index[0].unique())
            dof += num_order_paths - num_nonzero_outdegrees
    elif assumption == "ngram":
        # the ngram assumption corresponds to the assumption normally made when using higher-order markov chains
        # i.e., every node can follow every predecent sequence of nodes
        for order in range(1, max_order + 1):
            dof += (m.layers[1].data.num_nodes**order)*(m.layers[1].data.num_nodes - 1) 
    else:
        assert False, f"Unknown assumption {assumption} in input. The only accepted values are 'path' and 'ngram' "
    
    return int(dof)

get_mon_dof(m,max_order=1)

4

In [12]:
m.layers[1].data.num_nodes

5

In [14]:
from scipy.stats import chi2


def likeliHoodRatioTest(self, paths, maxOrderNull=0, maxOrder=1, assumption='paths', significanceThreshold=0.01):
        """
        Performs a likelihood-ratio test between two multi-order models with given maximum orders, where maxOrderNull serves
        as null hypothesis and maxOrder serves as alternative hypothesis. The null hypothesis is rejected if the p-value for
        the observed paths under the null hypothesis is smaller than the given significance threshold.

        Applying this test makes the assumption that we have nested models, i.e. that the null model is contained
        as a special case in the parameter space of the more complex model. If we assume that the path constraint holds,
        this is not true for the test of the first- against the zero-order model (since some sequences of the zero order model
        cannot be generated in the first-order model). However, since the set of possible higher-order transitions is generated
        based on the first-order model, the nestedness property holds for all higher order models.

        @param paths: the path data to be used in the liklihood ratio test
        @param maxOrderNull: maximum order of the multi-order model
                to be used as a null hypothesis
        @param maxOrder: maximum order of the multi-order model to be used as
                alternative hypothesis
        @param assumption: paths or ngrams
        @param significanceThreshold: the threshold for the p-value
                below which to accept the alternative hypothesis
        @returns: a tuple of the format (reject, p) which captures whether or
                not the null hypothesis is rejected in favor of the alternative
                hypothesis, as well as the p-value that led to the decision
        """

        assert maxOrderNull < maxOrder, 'Error: order of null hypothesis must be smaller than order of alternative hypothesis'
        # let L0 be the likelihood for the null model and L1 be the likelihood for the alternative model

        # we first compute a test statistic x = -2 * log (L0/L1) = -2 * (log L0 - log L1)
        x = -2 * (self.getLikelihood(paths, maxOrder=maxOrderNull, log=True) - self.getLikelihood(paths, maxOrder=maxOrder, log=True))

        # we calculate the additional degrees of freedom in the alternative model
        dof_diff = self.getDegreesOfFreedom(maxOrder=maxOrder, assumption=assumption) - self.getDegreesOfFreedom(maxOrder=maxOrderNull, assumption=assumption)

        Log.add('Likelihood ratio test for K_opt = ' + str(maxOrder) + ', x = ' + str(x))
        Log.add('Likelihood ratio test, d_1-d_0 = ' + str(dof_diff))

        # if the p-value is *below* the significance threshold, we reject the null hypothesis
        p = 1-chi2.cdf(x, dof_diff)

        Log.add('Likelihood ratio test, p = ' + str(p))
        return (p<significanceThreshold), p


    def estimateOrder(self, paths, maxOrder=None, significanceThreshold=0.01):
        """
        Selects the optimal maximum order of a multi-order network model for the
        observed paths, based on a likelihood ratio test with p-value threshold of p
        By default, all orders up to the maximum order of the multi-order model will be tested.

        @param paths: The path statistics for which to perform the order selection

        @param maxOrder: The maximum order up to which the multi-order model shall be tested.
        """
        if maxOrder == None:
            maxOrder = self.maxOrder
        assert maxOrder <= self.maxOrder, 'Error: maxOrder cannot be larger than maximum order of multi-order network'
        assert maxOrder > 1, 'Error: maxOrder must be larger than one'

        maxAcceptedOrder = 1

        # Test for highest order that passes
        # likelihood ratio test against null model
        for k in range(2, maxOrder+1):
            if self.likeliHoodRatioTest(paths, maxOrderNull=k-1, maxOrder=k, significanceThreshold=significanceThreshold)[0]:
                maxAcceptedOrder = k

        return maxAcceptedOrder

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 48)